In [1]:
import sys
sys.path.insert(1,'./decoders')
sys.path.insert(1,'./Instances_models')
sys.path.insert(1,'./Evaluators')
from TSP_decoder import TSPDecoder
from TSP_instance import TSPInstance
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
instance  = TSPInstance(r'C:\Users\Felipe\Projetos\Generative models optimization\Generative-model-optimization\Instances_data_TSP\burma14.dat')
decoder = TSPDecoder(instance)

In [4]:
print(instance.distances)

[153.0, 510.0, 706.0, 966.0, 581.0, 455.0, 70.0, 160.0, 372.0, 157.0, 567.0, 342.0, 398.0, 422.0, 664.0, 997.0, 598.0, 507.0, 197.0, 311.0, 479.0, 310.0, 581.0, 417.0, 376.0, 289.0, 744.0, 390.0, 437.0, 491.0, 645.0, 880.0, 618.0, 374.0, 455.0, 211.0, 491.0, 265.0, 410.0, 664.0, 804.0, 1070.0, 768.0, 259.0, 499.0, 310.0, 400.0, 514.0, 902.0, 990.0, 1261.0, 947.0, 418.0, 635.0, 636.0, 168.0, 522.0, 634.0, 910.0, 593.0, 19.0, 284.0, 239.0, 389.0, 482.0, 757.0, 439.0, 163.0, 124.0, 232.0, 154.0, 406.0, 133.0, 508.0, 273.0, 355.0, 276.0, 43.0, 623.0, 358.0, 498.0, 318.0, 898.0, 633.0, 761.0, 582.0, 315.0, 464.0, 275.0, 221.0, 247.0]
